# 006: Operation Graphs - Academic Claim Validation

Building on ReAct patterns from tutorial 005, this demonstrates sequential coordination using Operation Graphs to orchestrate ReaderTool workflows for academic claim validation.

## What You'll Learn

1. **Sequential Coordination**: Building context step-by-step through dependent operations
2. **ReaderTool Integration**: Document chunking and progressive reading strategies  
3. **Structured Extraction**: Using Pydantic models for reliable claim extraction
4. **Operation Dependencies**: How operations build on previous results

## Use Case: Validating a Theoretical Framework Paper

We'll validate claims in an academic paper about capability-based security by:
- Reading document chunks progressively with ReaderTool
- Building understanding through sequential analysis
- Extracting verifiable claims with structured output formats
- Demonstrating how each operation builds on the previous one's results

In [1]:
# Setup and imports
from typing import Literal
from pathlib import Path
from pydantic import BaseModel, Field

from lionagi import Branch, Session, Builder, types, iModel
from lionagi.tools.types import ReaderTool

# Target document - complex theoretical framework
here = Path().cwd()
document_path = here / "data" / "006_lion_proof_ch2.md"

print("✅ Environment setup complete")
print(f"📄 Target: {document_path.name}")
print("🎯 Goal: Validate academic claims using coordinated ReAct workflows")

✅ Environment setup complete
📄 Target: 006_lion_proof_ch2.md
🎯 Goal: Validate academic claims using coordinated ReAct workflows


In [2]:
# Data models for structured responses
class Claim(BaseModel):
    claim: str
    type: Literal["citation", "performance", "technical", "other"]
    location: str = Field(..., description="Section/paragraph reference")
    verifiability: Literal["high", "medium", "low"]
    search_strategy: str = Field(..., description="How to verify this claim")


class ClaimExtraction(BaseModel):
    claims: list[Claim]


print("✅ Data models defined")

✅ Data models defined


## Pattern 1: Sequential Document Analysis

Build understanding step-by-step: Open → Analyze → Extract claims

In [3]:
async def sequential_analysis():
    """Sequential workflow: open → analyze structure → extract claims."""

    # Create branch with ReaderTool
    branch = Branch(
        tools=[ReaderTool], chat_model=iModel(model="openai/gpt-4.1-mini")
    )
    session = Session(default_branch=branch)
    builder = Builder()

    # Step 1: Open and understand document
    doc_reader = builder.add_operation(
        "ReAct",
        node_id="open_document",
        instruct=types.Instruct(
            instruction="Use ReaderTool to open and analyze the theoretical framework document. Understand its structure and identify sections containing verifiable claims.",
            context={"document_path": str(document_path)},
        ),
        tools=["reader_tool"],
        max_extensions=2,
        verbose=True,
        verbose_length=200,
    )

    # Step 2: Progressive content analysis
    content_analyzer = builder.add_operation(
        "ReAct",
        node_id="analyze_content",
        depends_on=[doc_reader],
        instruct=types.Instruct(
            instruction="Read through key sections to identify citations, technical claims, and performance metrics that can be verified."
        ),
        response_format=types.Outline,
        tools=["reader_tool"],
        max_extensions=3,
        verbose=True,
        verbose_length=200,
    )

    # Step 3: Extract specific claims
    claim_extractor = builder.add_operation(
        "ReAct",
        node_id="extract_claims",
        depends_on=[content_analyzer],
        instruct=types.Instruct(
            instruction="Extract 5-7 specific, verifiable claims. Prioritize citations, performance metrics, and technical assertions."
        ),
        response_format=ClaimExtraction,
        tools=["reader_tool"],
        max_extensions=3,
        verbose=True,
        verbose_length=200,
    )

    # Execute workflow
    graph = builder.get_graph()
    print("🔗 Executing sequential analysis...")

    result = await session.flow(graph, parallel=False, verbose=True)

    return result


# Execute sequential analysis
result = await sequential_analysis()

🔗 Executing sequential analysis...
Executing operation: eaa2dd33-fb60-435c-ab9b-c6991e4a8fcc
Invoking action reader_tool with {'action': 'open', 'path_or_url': '/Users/lion/lio....
Action reader_tool invoked, status: completed.


### ReAct Round No.1 Analysis:
```yaml
analysis: I will first open the theoretical framework document to analyze its overall structure. After opening, I will read through the main sections to detect which parts contain verifiable c...

[Truncated output]
```
---------

### ReAct Final Answer:
```yaml

```
---------

Executing operation: 2b1894b1-8a11-4c7d-8107-349bc0254cee
Invoking action reader_tool with {'action': 'read', 'doc_id': 'DOC_9221526026981911....
Action reader_tool invoked, status: completed.


### ReAct Round No.1 Analysis:
```yaml
analysis: Given the length of the document (52,767 characters), we need to selectively read portions likely to contain verifiable citations, technical claims, or performance results. Typically...

[Truncated output]
```
---------

...

### ReAct Round No.5 Analysis:
```yaml
analysis: The last chunk read completes the proof of Theorem 2.1, including its three supporting lemmas which cover unforgeable capability references via WebAssembly isolation, preservation of...

[Truncated output]
```
---------

### ReAct Final Answer:
```yaml
topic: Analysis of Theoretical Framework Document for Verifiable Citations, Technical Claims, and Performance Metrics
items:
  - heading: 2.1 Introduction
    summary: Introduces the Lion ecos...

[Truncated output]
```
---------

Executing operation: b28df347-71f2-476c-8e7f-9f6740d4c3c0


### ReAct Round No.1 Analysis:
```yaml
analysis: Based on the analysis of the theoretical framework document, six specific verifiable claims have been extracted, focusing on formal theorems, implementation details, and mechanized p...

[Truncated output]
```
---------

### ReAct Final Answer:
```yaml
claims:
  - claim: Theorem 2.1: Cross-Component Capability Flow Preservation states that capability authority is preserved across component boundaries, and capability references remain unforge...

[Truncated output]
```
---------

In [4]:
from IPython.display import display, Markdown

# Display results
for node_id, data in result["operation_results"].items():
    if isinstance(data, types.Outline):
        md_content = f"""
## 📄 Document Structure ({node_id})

**Topic:** {data.topic}

### Key Sections:
"""
        for item in data.items[:3]:  # Show first 3
            md_content += f"- **{item.heading}**: {item.summary}\n"

        display(Markdown(md_content))

    elif isinstance(data, ClaimExtraction):
        md_content = f"""
## 📑 Extracted Claims ({node_id})

Found **{len(data.claims)}** verifiable claims:

"""
        for i, claim in enumerate(data.claims, 1):
            md_content += f"""
### {i}. [{claim.type.upper()}] {claim.claim}

- **Location:** {claim.location}  
- **Verifiability:** {claim.verifiability}
- **Search Strategy:** {claim.search_strategy}

"""
        display(Markdown(md_content))

display(Markdown("## ✅ Sequential analysis completed"))


## 📄 Document Structure (2b1894b1-8a11-4c7d-8107-349bc0254cee)

**Topic:** Analysis of Theoretical Framework Document for Verifiable Citations, Technical Claims, and Performance Metrics

### Key Sections:
- **2.1 Introduction**: Introduces the Lion ecosystem's novel capability-based security approach, motivating the work and outlining four main theorems with formal proofs and their integration with mechanized models and Rust implementation. Contains key technical claims about security challenges addressed and innovative contributions.
- **2.2 System Model and Formal Definitions**: Defines the Lion ecosystem architecture and formalizes the capability system as a mathematical model with sets for capabilities, rights, objects, subjects, policies, isolation contexts, and communication functions. Introduces formal definitions of capabilities, authority, component composition, and security properties. These definitions contain precise, verifiable technical claims foundational to subsequent theorems.
- **2.3 Theorem 2.1: Cross-Component Capability Flow**: States and formally proves Theorem 2.1, claiming capability authority preservation and unforgeability across component boundaries. Proof is divided into three lemmas: WebAssembly isolation enforcing reference integrity, capability transfer protocol ensuring authority preservation (with Rust code snippets providing implementation details), and policy compliance during transfer. Also notes mechanization of proofs in TLA+. This section holds multiple verifiable technical claims and partial implementation evidence, though no explicit performance metrics.



## 📑 Extracted Claims (b28df347-71f2-476c-8e7f-9f6740d4c3c0)

Found **6** verifiable claims:


### 1. [TECHNICAL] Theorem 2.1: Cross-Component Capability Flow Preservation states that capability authority is preserved across component boundaries, and capability references remain unforgeable during inter-component communication.

- **Location:** Section 2.3, Theorem 2.1 and subsections 2.3.1–2.3.3  
- **Verifiability:** high
- **Search Strategy:** Verify formal definitions and proofs presented in the document, examine mechanized TLA+ model (Appendix A.2), and review formal lemma proofs and Rust code snippets demonstrating unforgeability and authority preservation.


### 2. [TECHNICAL] Lemma 2.1.1 demonstrates that WebAssembly isolation preserves capability reference integrity by separating host memory and sandboxed linear memory, using cryptographically secure, injective handles unforgeable by WebAssembly modules independently.

- **Location:** Section 2.3.2, Lemma 2.1.1  
- **Verifiability:** high
- **Search Strategy:** Review the formal memory separation model and cryptographic assumptions described, validate the injective and unforgeable properties of handles through the mathematical definitions and supporting proofs.


### 3. [TECHNICAL] The capability transfer protocol serializes capability objects into handles with HMAC signatures ensuring integrity during transfer, as shown in Rust code examples of serialize_capability and deserialize_capability functions within lion core.

- **Location:** Section 2.3.2, Lemma 2.1.2  
- **Verifiability:** high
- **Search Strategy:** Examine the provided Rust code snippets and cryptographic signature usage, confirm implementation correctness by matching with protocol descriptions and verifying signature enforcement.


### 4. [TECHNICAL] All capability transfers are mediated by the lion policy component which enforces that transfers only occur if allowed by policy, modeled formally as send(s1,s2,c) implying policy_allows(s1,s2,c), ensuring policy compliance.

- **Location:** Section 2.3.2, Lemma 2.1.3  
- **Verifiability:** medium
- **Search Strategy:** Review formal policy enforcement descriptions, analyze the lion policy component behavior, and verify policy checks during capability transfer via system code or documentation.


### 5. [TECHNICAL] Theorem 2.2: Security Composition Preservation asserts that composition of individually secure, compatible Lion components preserves overall system security, including unforgeable refs, authority confinement, least privilege, and policy compliance.

- **Location:** Section 2.4, Theorem 2.2 and subsections 2.4.1–2.4.2  
- **Verifiability:** high
- **Search Strategy:** Verify formal theorem statements and lemma proofs related to composition, assess logical reasoning provided, and validate through theoretical model or mechanization if available.


### 6. [CITATION] Mechanized proofs and model checking of the Lion ecosystem's capability flow and security properties are conducted using TLA+, specifically for Theorem 2.1 and related lemmas.

- **Location:** Section 2.3.3 Conclusion and Appendix A.2 (referenced)  
- **Verifiability:** high
- **Search Strategy:** Access the TLA+ models and model checking results referenced in Appendix A.2 to confirm correctness and coverage of the formal verification.



## ✅ Sequential analysis completed